In [1]:
import requests
import json
import requests
import pandas as pd
import matplotlib.pyplot as plt

import numpy as np
import geopandas as gpd
import requests
import hvplot.pandas

np.random.seed(42)

In [2]:
pd.options.display.max_columns = 999

# 1. Loading Data

## 1.1 Load LODES

In [66]:
# Specify the output CSV file path
output_csv_path = 'Data_git/LODES/Home_Job/'
ct_home_job_2021 = pd.read_csv(output_csv_path + 'HOME_JOB_2021.csv')
ct_home_job_2019 = pd.read_csv(output_csv_path + 'HOME_JOB_2019.csv')

In [24]:
ct_home_job_2021.head()

,w_geocode_ct,h_geocode_ct,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03
0,34017000200,34017000200,13,0,10,3,1,11,1,0,4,9
1,34017000200,34017000300,1,0,0,1,0,0,1,0,0,1
2,34017000200,34017000400,2,0,2,0,0,0,2,0,0,2
3,34017000200,34017000500,1,0,0,1,1,0,0,0,0,1
4,34017000200,34017000700,4,3,0,1,0,2,2,0,1,3


In [33]:
lod_ml = ct_home_job_2021[['w_geocode_ct','S000']]
lod_ml = lod_ml.rename(columns = {'S000': "job"})
lod_ml = lod_ml.groupby('w_geocode_ct')['job'].sum().reset_index()
lod_ml.head()

,w_geocode_ct,job
0,34017000200,78
1,34017000300,75
2,34017000400,712
3,34017000500,68
4,34017000600,150


## 1.2 Centroid OD

In [54]:
from shapely.geometry import Point
import pandas as pd

In [64]:
# We have processed the geographical data
# You can load the NYC Metro Region geographical data on census tract directly
metro_geojson_path = 'Data_git/Geo/metro.geojson'
metro_gdf = gpd.read_file(metro_geojson_path)
metro_gdf.head()

,GEOID,COUNTYFP,geometry
0,36061001501,061,"POLYGON ((981865.515 198455.903, 981935.654 19..."
1,36061001600,061,"POLYGON ((984941.947 199431.109, 985056.156 19..."
2,36061001800,061,"POLYGON ((985796.223 201370.479, 985808.416 20..."
3,36061002000,061,"POLYGON ((990149.045 201586.466, 990284.254 20..."
4,36061002201,061,"POLYGON ((988552.924 201677.942, 988733.850 20..."


In [110]:
metro_gdf['centroid'] = metro_gdf['geometry'].centroid
metro_gdf.head()

,GEOID,COUNTYFP,geometry,centroid
0,36061001501,061,"POLYGON ((981865.515 198455.903, 981935.654 19...",POINT (983800.739 197130.851)
1,36061001600,061,"POLYGON ((984941.947 199431.109, 985056.156 19...",POINT (986116.768 200118.951)
2,36061001800,061,"POLYGON ((985796.223 201370.479, 985808.416 20...",POINT (986788.960 201244.639)
3,36061002000,061,"POLYGON ((990149.045 201586.466, 990284.254 20...",POINT (991778.422 201349.682)
4,36061002201,061,"POLYGON ((988552.924 201677.942, 988733.850 20...",POINT (989282.797 201270.261)


In [136]:
od = pd.merge(ct_home_job_2021, metro_gdf, left_on='w_geocode_ct', right_on='GEOID', how='left')
od = od.rename(columns = {'centroid': "centroid_w"})
od = od[["w_geocode_ct","h_geocode_ct", "centroid_w"]]

od.head()

,w_geocode_ct,h_geocode_ct,centroid_w
0,34017000200,34017000200,POINT (971257.758 213719.041)
1,34017000200,34017000300,POINT (971257.758 213719.041)
2,34017000200,34017000400,POINT (971257.758 213719.041)
3,34017000200,34017000500,POINT (971257.758 213719.041)
4,34017000200,34017000700,POINT (971257.758 213719.041)


In [137]:
od2 = pd.merge(ct_home_job_2021, metro_gdf, left_on='h_geocode_ct', right_on='GEOID', how='left')
od2 = od2.rename(columns = {'centroid': "centroid_h"})
od2 = od2[["w_geocode_ct","h_geocode_ct", "centroid_h"]]
od2.head()

,w_geocode_ct,h_geocode_ct,centroid_h
0,34017000200,34017000200,POINT (971257.758 213719.041)
1,34017000200,34017000300,POINT (972459.098 212644.110)
2,34017000200,34017000400,POINT (968055.224 212445.827)
3,34017000200,34017000500,POINT (970028.592 212498.393)
4,34017000200,34017000700,POINT (971242.247 211122.304)


In [138]:
merged_gdf = pd.merge(od2, od, left_on = ['w_geocode_ct', 'h_geocode_ct'], right_on=['w_geocode_ct', 'h_geocode_ct'], how='left')

merged_gdf.head()

,w_geocode_ct,h_geocode_ct,centroid_h,centroid_w
0,34017000200,34017000200,POINT (971257.758 213719.041),POINT (971257.758 213719.041)
1,34017000200,34017000300,POINT (972459.098 212644.110),POINT (971257.758 213719.041)
2,34017000200,34017000400,POINT (968055.224 212445.827),POINT (971257.758 213719.041)
3,34017000200,34017000500,POINT (970028.592 212498.393),POINT (971257.758 213719.041)
4,34017000200,34017000700,POINT (971242.247 211122.304),POINT (971257.758 213719.041)


In [139]:
merged_gdf['centroid_distance'] = merged_gdf.apply(lambda row: row['centroid_h'].distance(row['centroid_w']), axis=1)

In [140]:
merged_gdf = merged_gdf.rename(columns = {'centroid_distance': "OD_dis"})
merged_gdf.head()

,w_geocode_ct,h_geocode_ct,centroid_h,centroid_w,OD_dis
0,34017000200,34017000200,POINT (971257.758 213719.041),POINT (971257.758 213719.041),0.000000
1,34017000200,34017000300,POINT (972459.098 212644.110),POINT (971257.758 213719.041),1612.046894
2,34017000200,34017000400,POINT (968055.224 212445.827),POINT (971257.758 213719.041),3446.344792
3,34017000200,34017000500,POINT (970028.592 212498.393),POINT (971257.758 213719.041),1732.290484
4,34017000200,34017000700,POINT (971242.247 211122.304),POINT (971257.758 213719.041),2596.783100


## 1.3 Load ACS

In [14]:
data = pd.read_csv('Data_git/CSV/acs21_data.csv')

In [15]:
data.head()

,GEOID,COUNTYFP,geometry,area,NAME,21 population,21 total householder,21 owner householder,21 renter householder,21 meidan housing value,state,county,tract,pop_density,own_rate,rent_rate
0,36061001501,61,POLYGON ((981865.5151276055 198455.90345435744...,0.761081,"Census Tract 15.01, New York County, New York",7749,7290,4254,3036,767300,36,61,1501,10181.574978,0.583539,0.416461
1,36061001600,61,"POLYGON ((984941.9468926088 199431.108577446, ...",0.361350,"Census Tract 16, New York County, New York",8133,7950,1263,6687,668200,36,61,1600,22507.261055,0.158868,0.841132
2,36061001800,61,"POLYGON ((985796.222567258 201370.4793967813, ...",0.388537,"Census Tract 18, New York County, New York",7271,6666,635,6031,748100,36,61,1800,18713.780639,0.095260,0.904740
3,36061002000,61,POLYGON ((990149.0445261953 201586.46616041855...,0.460328,"Census Tract 20, New York County, New York",5015,4994,60,4934,-666666666,36,61,2000,10894.415408,0.012014,0.987986
4,36061002201,61,POLYGON ((988552.9238458973 201677.94219388932...,0.281718,"Census Tract 22.01, New York County, New York",6689,6521,847,5674,-666666666,36,61,2201,23743.599946,0.129888,0.870112


In [31]:
acs_ml = data[['GEOID','pop_density', 'own_rate', '21 meidan housing value','geometry']]
acs_ml = acs_ml[acs_ml['21 meidan housing value'] >= 0]
acs_ml = acs_ml.rename(columns = {"21 meidan housing value": "meidan_housing_value"})
acs_ml.head()

,GEOID,pop_density,own_rate,meidan_housing_value,geometry
0,36061001501,10181.574978,0.583539,767300,POLYGON ((981865.5151276055 198455.90345435744...
1,36061001600,22507.261055,0.158868,668200,"POLYGON ((984941.9468926088 199431.108577446, ..."
2,36061001800,18713.780639,0.095260,748100,"POLYGON ((985796.222567258 201370.4793967813, ..."
5,36061002202,24770.084463,0.052083,597800,"POLYGON ((988759.896978163 202177.4772083543, ..."
7,36061002601,22568.674204,0.132255,642600,"POLYGON ((989094.0677459151 202786.702712106, ..."


## 1.3 Combine and get the final dataset

The fianl dataset will include: 
- *ACS DATA:* pop_density, own_rate, meidan_housing_value
- *LODES DATA:* pop_density, own_rate, meidan_housing_value
- *Geo DATA:* GEOID

In [142]:
data_ml = pd.merge(acs_ml, lod_ml, left_on='GEOID', right_on='w_geocode_ct', how='left')
data_ml.head()

,GEOID,pop_density,own_rate,meidan_housing_value,geometry,w_geocode_ct,job
0,36061001501,10181.574978,0.583539,767300,POLYGON ((981865.5151276055 198455.90345435744...,36061001501,9956
1,36061001600,22507.261055,0.158868,668200,"POLYGON ((984941.9468926088 199431.108577446, ...",36061001600,3771
2,36061001800,18713.780639,0.095260,748100,"POLYGON ((985796.222567258 201370.4793967813, ...",36061001800,2455
3,36061002202,24770.084463,0.052083,597800,"POLYGON ((988759.896978163 202177.4772083543, ...",36061002202,210
4,36061002601,22568.674204,0.132255,642600,"POLYGON ((989094.0677459151 202786.702712106, ...",36061002601,457


In [143]:
data_ml = pd.merge(data_ml, merged_gdf, left_on='GEOID', right_on='w_geocode_ct', how='left')
data_ml.head()

,GEOID,pop_density,own_rate,meidan_housing_value,geometry,w_geocode_ct_x,job,w_geocode_ct_y,h_geocode_ct,centroid_h,centroid_w,OD_dis
0,36061001501,10181.574978,0.583539,767300,POLYGON ((981865.5151276055 198455.90345435744...,36061001501,9956,36061001501,36005000200,POINT (1022987.939 232964.272),POINT (983800.739 197130.851),53100.571168
1,36061001501,10181.574978,0.583539,767300,POLYGON ((981865.5151276055 198455.90345435744...,36061001501,9956,36061001501,36005000400,POINT (1025531.972 233828.849),POINT (983800.739 197130.851),55571.924571
2,36061001501,10181.574978,0.583539,767300,POLYGON ((981865.5151276055 198455.90345435744...,36061001501,9956,36061001501,36005001600,POINT (1023532.531 237637.297),POINT (983800.739 197130.851),56739.645205
3,36061001501,10181.574978,0.583539,767300,POLYGON ((981865.5151276055 198455.90345435744...,36061001501,9956,36061001501,36005001901,POINT (1004169.456 233219.806),POINT (983800.739 197130.851),41440.285363
4,36061001501,10181.574978,0.583539,767300,POLYGON ((981865.5151276055 198455.90345435744...,36061001501,9956,36061001501,36005001902,POINT (1007524.811 232091.571),POINT (983800.739 197130.851),42250.248490


# 2. Machine Learning

In our current project, our team employed a RandomForest model to delve into the essential features influencing OD_distance (Origin-Destination distance). This analysis aims to illuminate the intricate relationship between job locations and communities. Specifically, we intend to showcase the OD distance based on the working places, offering a clearer understanding of how job distribution correlates with community dynamics.

## 2.1 Train/Test & Features

In [179]:
# Models
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

# Model selection
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

# Pipelines
from sklearn.pipeline import make_pipeline

# Preprocessing
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

In [180]:
# Split the data 70/30
train_set, test_set = train_test_split(data_ml, test_size=0.3, random_state=42)

In [181]:
# Replace zero or negative values with a small positive value
epsilon = 1e-6
train_set['OD_dis'] = train_set['OD_dis'].apply(lambda x: epsilon if x <= 0 else x)
test_set['OD_dis'] = test_set['OD_dis'].apply(lambda x: epsilon if x <= 0 else x)

# Recalculate the log
y_train = np.log(train_set["OD_dis"])
y_test = np.log(test_set["OD_dis"])

In [152]:
# The features
feature_cols = [
   'pop_density', 
    'own_rate', 
    'meidan_housing_value',
    'job'
]
X_train = train_set[feature_cols].values
X_test = test_set[feature_cols].values

### Num & Cat Variable creating

In [182]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [183]:
# Set up a pipeline that includes both numerical columns and categorical columns
# Numerical columns
num_cols = [
   'pop_density', 
    'own_rate', 
    'meidan_housing_value',
    'job',
]

# Categorical columns
cat_cols = ["GEOID"]

### Start

In [187]:
# Set up the column transformer with two transformers
# ----> Scale the numerical columns
# ----> One-hot encode the categorical columns

transformer = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
    ]
)

In [188]:
# Initialize the pipeline
# NOTE: only use 10 estimators here so it will run in a reasonable time
pipe = make_pipeline(
    transformer, RandomForestRegressor(n_estimators=10, 
                                       random_state=42)
)

In [189]:
# Fit the training set
pipe.fit(train_set, y_train);

In [190]:
# What's the test score?
pipe.score(test_set, y_test)

0.11634595904963474

In [195]:
# The column transformer...
transformer

ColumnTransformer(transformers=[('num', StandardScaler(),
                                 ['pop_density', 'own_rate',
                                  'meidan_housing_value', 'job']),
                                ('cat', OneHotEncoder(handle_unknown='ignore'),
                                 ['GEOID'])])

In [196]:
# The steps in the column transformer
transformer.named_transformers_

{'num': StandardScaler(),
 'cat': OneHotEncoder(handle_unknown='ignore'),
 'remainder': 'drop'}

In [197]:
# The one-hot step
ohe = transformer.named_transformers_['cat']

ohe

OneHotEncoder(handle_unknown='ignore')

In [198]:
# One column for each category type!
ohe_cols = ohe.get_feature_names_out()

ohe_cols

array(['GEOID_34017000200', 'GEOID_34017000300', 'GEOID_34017000400', ...,
       'GEOID_36085031901', 'GEOID_36085031902', 'GEOID_36085032300'],
      dtype=object)

In [199]:
# Full list of columns is numerical + one-hot 
features = num_cols + list(ohe_cols)

features

['pop_density',
 'own_rate',
 'meidan_housing_value',
 'job',
 'GEOID_34017000200',
 'GEOID_34017000300',
 'GEOID_34017000400',
 'GEOID_34017000500',
 'GEOID_34017000600',
 'GEOID_34017000700',
 'GEOID_34017000800',
 'GEOID_34017000902',
 'GEOID_34017001000',
 'GEOID_34017001100',
 'GEOID_34017001201',
 'GEOID_34017001202',
 'GEOID_34017001300',
 'GEOID_34017001400',
 'GEOID_34017001701',
 'GEOID_34017001800',
 'GEOID_34017001900',
 'GEOID_34017002200',
 'GEOID_34017002300',
 'GEOID_34017002400',
 'GEOID_34017002700',
 'GEOID_34017002800',
 'GEOID_34017002900',
 'GEOID_34017003000',
 'GEOID_34017003500',
 'GEOID_34017004000',
 'GEOID_34017004102',
 'GEOID_34017004200',
 'GEOID_34017004300',
 'GEOID_34017004400',
 'GEOID_34017004500',
 'GEOID_34017004600',
 'GEOID_34017004700',
 'GEOID_34017004800',
 'GEOID_34017004900',
 'GEOID_34017005200',
 'GEOID_34017005300',
 'GEOID_34017005400',
 'GEOID_34017005500',
 'GEOID_34017005600',
 'GEOID_34017005801',
 'GEOID_34017005802',
 'GEOID_340170

In [200]:
random_forest = pipe["randomforestregressor"]

# Create the dataframe with importances
importance = pd.DataFrame(
    {"Feature": features, "Importance": random_forest.feature_importances_}
)

In [201]:
importance.head(n=20)

,Feature,Importance
0,pop_density,4.579829e-01
1,own_rate,4.867511e-02
2,meidan_housing_value,6.849942e-02
3,job,2.807947e-01
4,GEOID_34017000200,6.644968e-06
5,GEOID_34017000300,1.053141e-05
6,GEOID_34017000400,1.165587e-05
7,GEOID_34017000500,1.332629e-05
8,GEOID_34017000600,0.000000e+00
9,GEOID_34017000700,1.801546e-05


### Which variables were most important?

The first thing we want to know is to find which features are important to OD data.

In [203]:
# Extract the regressor from the pipeline
forest_model = forest_pipeline["randomforestregressor"]

In [210]:
importance = pd.DataFrame(
    {"Feature": features, "Importance": random_forest.feature_importances_}
).sort_values("Importance", ascending= False)

# Select the top five features
top_five_importance = importance.head(10)

# Create a horizontal bar plot using hvplot
top_five_importance.hvplot.barh(x="Feature", y="Importance", title="Top 5 Feature Importances")

:Bars   [Feature]   (Importance)

### Try to find the best model

In [212]:
model_name = "randomforestregressor"
param_grid = {
    f"{model_name}__n_estimators": [15, 30, 45],
    f"{model_name}__max_depth": [None, 10, 20],
}

param_grid

{'randomforestregressor__n_estimators': [15, 30, 45],
 'randomforestregressor__max_depth': [None, 10, 20]}

In [213]:
# Create the grid and use 5-fold CV (10-fold is too slow for my laptop)
grid = GridSearchCV(pipe, param_grid, cv=5, n_jobs=2)

In [ ]:
# Run the search
grid.fit(train_set, y_train);

In [ ]:
grid.best_params_

In [ ]:
# Evaluate the best random forest model
best_random = grid.best_estimator_
grid.score(test_set, y_test)

### 2.5 Calculate the percent error of your model predictions for each sale in the test set

In [ ]:
test_set.index

In [ ]:
# Extract the test data from the original dataset
# This will include the geometry data
X = data_ml.loc[test_set.index]

In [ ]:
# test data extracted from our original data frame
X.head()

In [ ]:
# Predictions for log of total trip counts
predictions = grid.best_estimator_.predict(test_set)

# Convert the predicted test values from log
percent_error = ((np.exp(y_test)-np.exp(predictions))/ (np.exp(y_test))) * 100

In [ ]:
X['predicted_sale_price'] = np.exp(predictions)
X['pct_error']  = percent_error

In [ ]:
X.head()